# Fishing enhancement data processing. 
This notebook loads and processes the fishing enhancement data to be used in the fishing enhancement widget, as part of the Mangrove Fisheries card.

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os

In [ ]:
cloud_data_path = "https://storage.googleapis.com/mangrove_atlas/widget_data/"

## Load and preview data

In [ ]:
df_original_abs = pd.read_csv(cloud_data_path + "COUNTRY_ABUNDANCE.csv") 
df_original_abs = df_original_abs[['Name', 'gid_0', 'Total_Enhance', 'Fish_Enhance', 'Crab_Enhance', 'Bivalve_Enhance', 'Shrimp_Enhance']]
df_original_abs.columns = ['Name', 'gid_0', 'Total', 'Fish', 'Crab', 'Bivalve', 'Shrimp']
df_original_abs.head()

,Name,gid_0,Total,Fish,Crab,Bivalve,Shrimp
0,American Samoa,ASM,3.223102e+05,3.205375e+05,1.772723e+03,0.0,0.000000e+00
1,Anguilla,AIA,5.556892e+04,3.024532e+04,2.532360e+04,0.0,0.000000e+00
2,Antigua and Barbuda,ATG,1.735705e+07,9.447177e+06,7.909870e+06,0.0,0.000000e+00
3,Aruba,ABW,6.577852e+05,3.580225e+05,2.997627e+05,0.0,0.000000e+00
4,Australia,AUS,2.387689e+10,7.655215e+09,4.091423e+07,0.0,1.618077e+10


In [ ]:
df_original_den = pd.read_csv(cloud_data_path + "COUNTRY_DENSITY.csv") 
df_original_den = df_original_den[['Name', 'gid_0', 'Total', 'Fish', 'Crab','Bivalve', 'Shrimp']]
df_original_den.head()

,Name,gid_0,Total,Fish,Crab,Bivalve,Shrimp
0,American Samoa,ASM,104.501501,103.926737,0.574764,0.0,0.000000
1,Anguilla,AIA,150.256135,81.782129,68.474005,0.0,0.000000
2,Antigua and Barbuda,ATG,199.471778,108.569455,90.902322,0.0,0.000000
3,Aruba,ABW,148.070295,80.592410,67.477885,0.0,0.000000
4,Australia,AUS,274.837110,87.834544,0.475007,0.0,186.527559


## Process data.  
Change to long format and combine tables.  
Adjust field names. 


In [ ]:
df_abs_long = pd.melt(df_original_abs, id_vars=['Name', 'gid_0'], var_name='indicator', value_name='value')
df_abs_long['type'] = 'absolute'
df_abs_long = df_abs_long[['Name', 'gid_0','type', 'indicator', 'value']]

df_den_long = pd.melt(df_original_den, id_vars=['Name', 'gid_0'], var_name='indicator', value_name='value')
df_den_long['type'] = 'density'
df_den_long = df_den_long[['Name', 'gid_0','type', 'indicator', 'value']]

df_fisheries = pd.concat([df_abs_long, df_den_long], ignore_index=True)
df_fisheries['indicator'] = df_fisheries['indicator'].str.lower()

df_fisheries.sample(10)

,Name,gid_0,type,indicator,value
868,Jamaica,JAM,density,bivalve,0.000000e+00
414,Antigua and Barbuda,ATG,absolute,shrimp,0.000000e+00
860,Guam,GUM,density,bivalve,0.000000e+00
235,Eritrea,ERI,absolute,crab,3.741512e+05
298,Turks and Caicos Islands,TCA,absolute,crab,7.739303e+07
465,Mauritius,MUS,absolute,shrimp,3.935230e+06
603,Thailand,THA,density,total,3.362856e+02
681,Pakistan,PAK,density,fish,6.260877e+02
251,Japan,JPN,absolute,crab,5.311628e+04
282,Saudi Arabia,SAU,absolute,crab,4.416849e+05


## Get locations data

In [ ]:
locations_file = 'https://storage.googleapis.com/mangrove_atlas/boundaries/processed/location_final/locations_v3_not_merged_with_old.gpkg'
locations = gpd.read_file(locations_file)
locations = locations[locations['type'] == 'country'][['iso', 'location_idn']]
locations.head()

,iso,location_idn
82,QAT,06d2e6f9-bc89-59bf-a0e2-ab804e5db9fd
89,MYT,0750953f-4af9-549b-aeea-329663249a56
118,VNM,09a1ab14-11ad-56ec-8acb-a149e5697abd
132,GRD,0b0ecb56-bb8e-5ef1-b8ee-3cdad67fed0e
149,IND,0c07ca53-7b17-5650-a2c6-0cc27249a4bd


In [ ]:
api_locs = pd.read_csv('https://storage.googleapis.com/mangrove_atlas/widget_data/locations_staging.csv')
api_locs.rename(columns={'location_id': 'location_idn'}, inplace=True)
api_locs.head()

,id,location_idn
0,1563,000bd204-c0fd-510b-a1ad-132a7ef7470d
1,1564,00250a0f-f66d-54a0-b7a3-d80035881cbf
2,1565,0041637b-f6a2-5b89-87ce-850f5c5431b3
3,1566,005b49ef-6b7f-575a-85b3-ff19261a0755
4,1567,00921349-70fb-5a7e-8207-b3157aecc349


In [ ]:
print(locations['iso'].nunique())
print(df_fisheries['gid_0'].nunique())
print(df_fisheries[~df_fisheries['gid_0'].isin(locations['iso'])].shape[0])


122
103
0


In [ ]:
df_fisheries_full = pd.merge(df_fisheries, locations, left_on='gid_0', right_on='iso', how='left')
df_fisheries_full = pd.merge(df_fisheries_full, api_locs, on='location_idn', how='left')
df_fisheries_full = df_fisheries_full[['id', 'Name', 'type', 'indicator', 'value']]
df_fisheries_full.rename(columns={'id': 'location_id', 'type':'indicator type'}, inplace=True)
df_fisheries_full.sample(10)

,location_id,Name,indicator type,indicator,value
453,1712,India,absolute,shrimp,2.915643e+10
484,3671,Saint Lucia,absolute,shrimp,0.000000e+00
215,3831,Belize,absolute,crab,3.722927e+08
873,4195,Malaysia,density,bivalve,0.000000e+00
386,4368,Seychelles,absolute,bivalve,0.000000e+00
293,4511,"Tanzania, United Republic of",absolute,crab,1.094364e+10
199,3248,Vanuatu,absolute,fish,1.481313e+07
33,3094,French Southern Territories,absolute,total,1.765219e+07
397,3760,Thailand,absolute,bivalve,0.000000e+00
847,2550,Djibouti,density,bivalve,0.000000e+00


In [ ]:
#Check single country
df_fisheries_full[df_fisheries_full['Name'] == 'India']

,location_id,Name,indicator type,indicator,value
41,1712,India,absolute,total,4.248683e+10
144,1712,India,absolute,fish,1.326361e+10
247,1712,India,absolute,crab,6.678909e+07
350,1712,India,absolute,bivalve,0.000000e+00
453,1712,India,absolute,shrimp,2.915643e+10
556,1712,India,density,total,9.607643e+02
659,1712,India,density,fish,3.008290e+02
762,1712,India,density,crab,1.513732e+00
865,1712,India,density,bivalve,0.000000e+00
968,1712,India,density,shrimp,6.584216e+02


In [ ]:
# Check missing values
df_fisheries_full[df_fisheries_full['value'].isna()]

,location_id,Name,indicator type,indicator,value
547,3862,French Polynesia,density,total,NaN
650,3862,French Polynesia,density,fish,NaN
753,3862,French Polynesia,density,crab,NaN
856,3862,French Polynesia,density,bivalve,NaN
959,3862,French Polynesia,density,shrimp,NaN


## Export final data

In [ ]:
df_final = df_fisheries_full.copy()
df_final.drop(columns=['Name'], inplace=True)
df_final = df_final[df_final['value'].notna()]
#df_final = df_final[df_final['value'] > 0]
df_final = df_final[df_final['indicator'] != 'total']
df_final.to_csv('../../../../data/fisheries_enhancement_UPLOAD.csv', index=False)